<a href="https://colab.research.google.com/github/sushanttwayana/Text2HumanFace_Generation/blob/main/Celeba_Dataset_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'celeba-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F29561%2F37705%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T181210Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D572efabecf4823a42cb664573935c6fabe855cf5a1841f5df47352880eb571146c1d9931bf69c5b67335f2e6dfdcc028b579a526e2a006dfa718842eef147c431d99c0d0a5f45be72821b00fc32e8aba52f00033169ddf7bebf57d46d9fa4eee8aef061756bcd52bbffcda65452ae74c909d399dc1719b5becd32b212a1f8174f8967a022583e3de9841f3b48710a1c4af8bed6058e8c796869ad79fd78d39fcfe70acfa865359a343f5978e9f2eda82327d89e1a05c86bc6049597a89a1f9a5b7eb5f6d60b1bed11b1893a99aaf2fa2daee6010281a76b3419432259afda601544e5738105369641e118af23a156d0fa7fc1da97c084ec5e3e174ed7101f269,celeba-text:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4944262%2F8323379%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T181210Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7745a3f9826cb33a654bb18c754e8ded5bdea08f5620866b7d1fd8741d994dd587f7eefa45dabb8d0fb367d41c59f520b024cfb199511f52f6d2c7b3709c63f70b127282ce48484fd6a9aabd411d5f8af5b4610a2485d105d88264686fdb92af7559bc25c91cd66f7a71804b89896eb11ece2a0d61427203487e9c03db7e7116554f545df16555c704105a9312dcdb387b93a2b0d4ac471dd3c9b4bf36bef6b40b3c4e8178f1c8317ba90cc9541f0540d64f5691f9ffe127825ec3c9ba998dce224c695b5d51a33471e4a675879fd462b8ce4219983043b6b9ecb39c2008db1b8ea8fee38b5a51d394d4075abe594ceb403781aca6ee928abea25e16d9d2a46e,emotionaldataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5204052%2F8695562%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T181210Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3eb2972e4cefd7bc1924f074445d8755490b10448ded9ceb85554b8bcde7393c2168388f139c68e4fe84a67d3631c6a8f00321d173348f96bf92b78f5a77cdc4c92e2c8109bbb9d00c411304568362072550028ce7ba8f3d5c95f7e7c21fe583da296704c072074fd38c587b0a904d81c48062428e2e66f3e4bbc6871aff9b844bcf75e3bb270d9352fefbc2455ab81c914d4d3d2a296ae6f220f83e6b5599df695203f98d7c0c185d18f184f034398d6c9f5189195aab0daa4a51528831b6c8a8ccda56e4be9ae2ea7cba5454ddb0d902056331e2a9b2ccf143fdc5e6958dcdcd65eb62f3786159f45627bf4fbfc5544b41dcc4d1e543a8e2532206ea5947c0,final-celeba-12000:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5227713%2F8713758%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240617%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240617T181211Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D690b5c560e1972ca1dc8d0e673c8747be0989b1ffaa078fa2e75bd421fef7b0ef36c5552f60a8517decf4acbb6d7cc563b660acc629b03788d8acc5237aa7bebbcd82b56783dea88dc2595b4b769e3c80c7907d527694b77a165c84a1470c096c0753416c99ea3b46e863015d282675d7c1bc93dd88ba6a4464e30e145b6fc1bf066594ba0963a76683a3abb4121423e5d86c5194a98ae4b6586d970070fc3e0aa357d89b53dad66b282171698f746f062df605d66abe5f06fef3d81c21bdd87cea17989ff132b2dd74dacefa28cc2cf093412751b116396a1793be483a23ce0e9b8ab71b822f9b12cc05688915680cf9c68f0eb757c7cfad326eb387d966258'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import json

# Load the JSON data from the file
with open('/kaggle/working/final_sorted_csv_120000..json', 'r') as json_file:
    data = json.load(json_file)

# Define the desired text format
text = ''

# Process each item in the list
for item in data:
    text += f"{item}\n"  # Assuming each item is a string representation

# Write the text to a new file
with open('/kaggle/working/final_celeba_120000.txt', 'w') as text_file:
    text_file.write(text)


In [ ]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    data = []
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            data.append(row)

    with open(json_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# Example usage
csv_to_json('/kaggle/input/final-celeba-12000/final_sorted_csv_120000.csv', '/kaggle/working/final_sorted_csv_120000..json')

In [ ]:
import os
import csv

def generate_captions_from_attributes(attributes_file, output_file, exclude_attributes):
    captions = []

    # Read the attributes from the file
    with open(attributes_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            image_id = row['image_id']

            # Convert attributes to textual description
            description = []
            gender = "male" if row['Male'] == '1' else "female"
            description.append(f"This person is {gender}")

            pronoun = "He" if gender == "male" else "She"
            possessive_pronoun = "his" if gender == "male" else "her"

            # Include attributes in the description
            for attribute, value in row.items():
                if attribute == 'image_id' or attribute == 'Male':
                    continue
                if exclude_attributes and attribute in exclude_attributes:
                    continue

                if value == '1':
                    if attribute == '5_o_Clock_Shadow':
                        description.append(f"{pronoun} has dense beard.")
                    elif attribute == 'Arched_Eyebrows':
                        description.append(f"{pronoun} has arched eyebrows.")
                    elif attribute == 'Attractive':
                        description.append(f"{pronoun} is attractive.")
                    elif attribute == 'Bags_Under_Eyes':
                        description.append(f"{pronoun} has bags under {possessive_pronoun} eyes.")
                    elif attribute == 'Bald':
                        description.append(f"{pronoun} is bald.")
                    elif attribute == 'Bangs':
                        description.append(f"{pronoun} has bangs.")
                    elif attribute == 'Big_Lips':
                        description.append(f"{pronoun} has big lips.")
                    elif attribute == 'Big_Nose':
                        description.append(f"{pronoun} has a big nose.")
                    elif attribute == 'Black_Hair':
                        description.append(f"{pronoun} has black hair.")
                    elif attribute == 'Blond_Hair':
                        description.append(f"{pronoun} has blonde hair.")
                    elif attribute == 'Brown_Hair':
                        description.append(f"{pronoun} has brown hair.")
                    elif attribute == 'Bushy_Eyebrows':
                        description.append(f"{pronoun} has bushy eyebrows.")
                    elif attribute == 'Chubby':
                        description.append(f"{pronoun} is chubby.")
                    elif attribute == 'Double_Chin':
                        description.append(f"{pronoun} has a double chin.")
                    elif attribute == 'Eyeglasses':
                        description.append(f"{pronoun} is wearing eyeglasses.")
                    elif attribute == 'Goatee':
                        description.append(f"{pronoun} has a goatee beard.")
                    elif attribute == 'Gray_Hair':
                        description.append(f"{pronoun} has gray hair.")
                    elif attribute == 'Heavy_Makeup':
                        description.append(f"{pronoun} is wearing heavy makeup.")
                    elif attribute == 'High_Cheekbones':
                        description.append(f"{pronoun} has high cheekbones.")
                    elif attribute == 'Mouth_Slightly_Open':
                        description.append(f"{pronoun} has {possessive_pronoun} mouth slightly open.")
                    elif attribute == 'Mustache':
                        description.append(f"{pronoun} has a mustache.")
                    elif attribute == 'Narrow_Eyes':
                        description.append(f"{pronoun} has narrow eyes.")
                    elif attribute == 'Oval_Face':
                        description.append(f"{pronoun} has an oval face.")
                    elif attribute == 'Pale_Skin':
                        description.append(f"{pronoun} has pale skin.")
                    elif attribute == 'Pointy_Nose':
                        description.append(f"{pronoun} has a pointy nose.")
                    elif attribute == 'Receding_Hairline':
                        description.append(f"{pronoun} has a receding hairline.")
                    elif attribute == 'Rosy_Cheeks':
                        description.append(f"{pronoun} has rosy cheeks.")
                    elif attribute == 'Sideburns':
                        description.append(f"{pronoun} has sideburns.")
                    elif attribute == 'Smiling':
                        description.append(f"{pronoun} is smiling.")
                    elif attribute == 'Straight_Hair':
                        description.append(f"{pronoun} has straight hair.")
                    elif attribute == 'Wavy_Hair':
                        description.append(f"{pronoun} has wavy hair.")
                    elif attribute == 'Wearing_Earrings':
                        description.append(f"{pronoun} is wearing earrings.")
                    elif attribute == 'Wearing_Hat':
                        description.append(f"{pronoun} is wearing a hat.")
                    elif attribute == 'Wearing_Lipstick':
                        description.append(f"{pronoun} is wearing lipstick.")
                    elif attribute == 'Wearing_Necklace':
                        description.append(f"{pronoun} is wearing a necklace.")
                    elif attribute == 'Wearing_Necktie':
                        description.append(f"{pronoun} is wearing a necktie.")
                    elif attribute == 'Young':
                        description.append(f"{pronoun} is young.")
                    elif attribute == "happy":
                        description.append(f"{pronoun} is happy.")
                        description.append(f"{pronoun} looks happy.")
                        description.append(f"{pronoun} is joyful and delighted.")
                        description.append(f"{pronoun} has a happy expression.")
                        description.append(f"{pronoun} seems blissful.")
                        description.append(f"{pronoun} is cheerful.")
                    elif attribute == "sad":
                        description.append(f"{pronoun} is sad.")
                        description.append(f"{pronoun} looks sad.")
                        description.append(f"{pronoun} looks unhappy and sad.")
                        description.append(f"{pronoun} looks depressed.")
                        description.append(f"{pronoun} is feeling down.")
                        description.append(f"{pronoun} has a sad expression.")
                    elif attribute == "neutral":
                        description.append(f"{pronoun} is neutral.")
                        description.append(f"{pronoun} stays calm and neutral.")
                        description.append(f"{pronoun} stays open-minded and is fair-minded.")

                elif value == '-1':
                    if attribute == 'No_Beard':
                        description.append(f"{pronoun} has a beard.")
                    elif attribute == 'Young':
                        description.append(f"{pronoun} is old.")
                    elif attribute == 'Pale_Skin':
                        description.append(f"{pronoun} has brown skin.")
                    elif attribute == 'Chubby':
                        description.append(f"{pronoun} is slim.")
                    elif attribute == 'Bald':
                        description.append(f"{pronoun} has hair.")
                    elif attribute == 'Big_Lips':
                        description.append(f"{pronoun} has thin lips.")
                    elif attribute == 'Big_Nose':
                        description.append(f"{pronoun} has a small nose.")
                    elif attribute == 'Mustache':
                        if row['Male'] == '1':
                            description.append(f"{pronoun} is clean-shaven and has no facial hairs.")
                        else :
                            description.append(f"{pronoun} has no facial hairs.")
                    elif attribute == 'Oval_Face':
                        description.append(f"{pronoun} has a round face.")
                    elif attribute == 'Straight_Hair':
                        description.append(f"{pronoun} has curly hair.")
                    elif attribute == 'Sideburns':
                        description.append(f"{pronoun} has bare cheeks. {pronoun} has no facial hair on the sides.")
                    elif attribute == 'Pointy_Nose':
                        description.append(f"{pronoun} has a rounded nose.")
                    elif attribute == 'High_Cheekbones':
                        description.append(f"{pronoun} has flat cheekbones.")
                    elif attribute == 'Heavy_Makeup':
                        description.append(f"{pronoun} is wearing no makeup. {pronoun} is wearing light makeup.")
                    elif attribute == 'Smiling':
                        description.append(f"{pronoun} is not smiling.")
                    elif attribute == 'Mouth_Slightly_Open':
                        description.append(f"{pronoun} has {possessive_pronoun} mouth closed.")

            captions.append((image_id, ' '.join(description)))

    # Save captions to output file
    if not os.path.exists(os.path.dirname(output_file)):
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with open(output_file, 'w') as file:
        file.write("image,caption\n")  # Write the header line
        for image_id, description in captions:
            file.write(f"{image_id},{description}\n")

# Example usage:
attributes_file = '/kaggle/input/final-celeba-12000/final_sorted_csv_120000.csv'
output_file = '/kaggle/working/descriptions/celeba_120000_captions.txt'
exclude_attributes = ['Blurry']  # Example of attributes to exclude
generate_captions_from_attributes(attributes_file, output_file, exclude_attributes)


In [ ]:
# from keras.utils.np_utils import to_categorical
# import pandas as pd
# import numpy as np
# import random
# import sys
# import warnings
# warnings.filterwarnings('ignore')

# import matplotlib.pyplot as plt
# %matplotlib inline

# import brewer2mpl


# def emotion_count(y_train, classes):
#     """
#     The function re-classify picture with disgust label into angry label
#     """
#     emo_classcount = {}
#     print ('Disgust classified as Angry')
#     y_train.loc[y_train == 1] = 0
#     classes.remove('Disgust')
#     for new_num, _class in enumerate(classes):
#         y_train.loc[(y_train == emotion[_class])] = new_num
#         class_count = sum(y_train == (new_num))
#         emo_classcount[_class] = (new_num, class_count)
#     return y_train.values, emo_classcount

# def load_data(sample_split=0.3, usage='Training',classes=['Angry','Happy'], filepath='../input/fer20131.csv'):
#     """
#     The function load provided CSV dataset and further reshape, rescale the data for feeding
#     """
#     df = pd.read_csv(filepath)
#     df = df[df.Usage == usage]
#     frames = []
#     classes.append('Disgust')
#     for _class in classes:
#         class_df = df[df['emotion'] == emotion[_class]]
#         frames.append(class_df)
#     data = pd.concat(frames, axis=0)
#     rows = random.sample(list(data.index), int(len(data)*sample_split))
#     data = data.loc[rows]
#     x = list(data["pixels"])
#     X = []
#     for i in range(len(x)):
#         each_pixel = [int(num) for num in x[i].split()]
#         X.append(each_pixel)
#     ## reshape into 48*48*1 and rescale
#     X = np.array(X)
#     X = X.reshape(X.shape[0], 48, 48,1)
#     X = X.astype("float32")
#     X /= 255

#     y_train, new_dict = emotion_count(data.emotion, classes)
#     y_train = to_categorical(y_train)
#     return X, y_train

# Emotional Dataset Generation

In [ ]:
pip install tensorflow

In [ ]:
model_path ='/kaggle/input/your-dataset/your_model.h5'

In [ ]:
import h5py
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model('/kaggle/input/emotionaldataset/model.h5')


In [ ]:
loaded_model.summary()

In [ ]:
from PIL import Image

# Load the image
image_path = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/000050.jpg'  # Replace with your image file path
image = Image.open(image_path)

In [ ]:
# Resize the image to 48x48
resized_image = image.resize((48, 48))

# Convert the resized image to numpy array (optional)
import numpy as np
resized_image_array = np.array(resized_image)


In [ ]:
resized_image_gray = resized_image.convert('L')

In [ ]:
import matplotlib.pyplot as plt

# Display the grayscale image using matplotlib
plt.imshow(resized_image_gray, cmap='gray')
plt.axis('off')  # Hide axes for better visualization
plt.show()

In [ ]:
# Convert the resized image to a format suitable for model input
import tensorflow as tf
resized_image_array = np.expand_dims(resized_image_gray, axis=0)  # Add batch dimension
resized_image_array = np.expand_dims(resized_image_array, axis=-1)  # Add channels dimension (for grayscale)
resized_image_array = resized_image_array.astype('float32') / 255.0  # Normalize pixel values


In [ ]:
# prediction and true labels
y_prob = loaded_model.predict(resized_image_array, batch_size=1, verbose=0)
y_pred = [np.argmax(prob) for prob in y_prob]

print(y_prob)
print(y_pred)

In [ ]:
mapper = {
    0: "happy",
    1: "sad",
    2: "neutral",
}

emo     = ["happy", 'sad', 'neutral']

# Using list comprehension to retrieve elements
result = [emo[idx] for idx in y_pred]

print(result)

In [ ]:
import os
import csv
from PIL import Image
import numpy as np
import tensorflow as tf

# Define the mapper and emotions
mapper = {
    0: "happy",
    1: "sad",
    2: "neutral",
}

emo = ["happy", "sad", "neutral"]

# Load your pre-trained model
# loaded_model = tf.keras.models.load_model('/kaggle/input/emotionaldataset/model.h5')

# Define the directory containing tahe images
image_directory = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba'

# Prepare a list to store results
results = []

# Iterate over each image in the directory
for image_name in os.listdir(image_directory):
    if image_name.endswith('.jpg'):  # Add other image formats if needed
        image_path = os.path.join(image_directory, image_name)

        # Load and process the image
        image = Image.open(image_path)
        resized_image = image.resize((48, 48))
        resized_image_gray = resized_image.convert('L')
        resized_image_array = np.expand_dims(resized_image_gray, axis=0)  # Add batch dimension
        resized_image_array = np.expand_dims(resized_image_array, axis=-1)  # Add channels dimension (for grayscale)
        resized_image_array = resized_image_array.astype('float32') / 255.0  # Normalize pixel values

        # Make a prediction
        y_prob = loaded_model.predict(resized_image_array, batch_size=1, verbose=0)
        y_pred = np.argmax(y_prob)

        # Get the corresponding emotion label
        emotion_label = mapper[y_pred]

        # Append the result to the list
        results.append([image_name, emotion_label])

# Save the results to a CSV file
csv_file_path = '/kaggle/working/results.csv'
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Emotion'])
    writer.writerows(results)

print(f'Results saved to {csv_file_path}')


# Merging two csv files

In [ ]:
import pandas as pd

# Load the original CSV file with appropriate settings
original_csv = '/kaggle/input/celeba-dataset/list_attr_celeba.csv'
df_original = pd.read_csv(original_csv, index_col=0)

# Load the second CSV file with new attributes
new_csv = 'path_to_your_new_csv_file.csv'  # Replace with the actual path to your new CSV file
df_new = pd.read_csv(new_csv)

In [ ]:
# Add the new columns to the original dataframe and initialize them with -1
df_original['happy'] = -1
df_original['sad'] = -1
df_original['neutral'] = -1

In [ ]:
# Update the new columns based on the second CSV file
for index, row in df_new.iterrows():
    image_id = row['image_number']  # Assuming the column in new CSV is named 'image_number'
    attribute = row['attribute']  # This should be one of 'happy', 'sad', 'neutral'

    # Set the corresponding attribute column to 1
    df_original.loc[image_id, attribute] = 1

# Save the updated dataframe to a new CSV file
df_original.to_csv('updated_celeba.csv')

In [ ]:
import pandas as pd
import csv

# Load the original and new CSV files
df_original = pd.read_csv('/kaggle/input/celeba-dataset/list_attr_celeba.csv')
df_new = pd.read_csv('/kaggle/input/emotionaldataset/results (1).csv')

# Print column names for debugging purposes
print(df_new.columns)

# Assuming the column in the new CSV is named 'Image' for image_id and 'Emotion' for attribute
for index, row in df_new.iterrows():
    image_id = row['Image']  # Column name in the new CSV
    attribute = row['Emotion']  # Column name in the new CSV ('happy', 'sad', 'neutral')

    # Ensure the original DataFrame has the attribute columns
    if attribute not in df_original.columns:
        df_original[attribute] = 0

    # Set the corresponding attribute column to 1
    df_original.loc[df_original['image_id'] == image_id, attribute] = 1

# Save the updated dataframe to a new CSV file
df_original.to_csv('/kaggle/working/final_updated_celeba.csv', index=False)

print(f'Updated CSV saved to /kaggle/working/updated_celeba.csv')


**Sort in Ascending Order**

In [ ]:
import pandas as pd

# Step 1: Read the CSV file
file_path = '/kaggle/input/emotionaldataset/emotion.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Step 2: Sort the DataFrame based on the 'image' column
df_sorted = df.sort_values(by='Image')

# Step 3: Save the sorted DataFrame to a new CSV file
sorted_file_path = '/kaggle/working/sorted_file.csv'  # Replace with your desired output file path
df_sorted.to_csv(sorted_file_path, index=False)

print(f"Sorted CSV file has been saved to {sorted_file_path}")


In [ ]:
# Step 1: Read the CSV file
file_path = '/kaggle/input/emotionaldataset/results (1).csv'  # Replace with your actual file path
df2 = pd.read_csv(file_path)

# Step 2: Count the number of images
number_of_images = len(df['Image'])

print(f"Number of images in the CSV file: {number_of_images}")